# This program can plot u_e_ij for a $1 \times M$ mesh.

In [1]:
import os
import sys
sys.path.insert(0, os.path.abspath('../../'))

from matplotlib import pyplot as plt
import numpy as np
import arrayfire as af
import h5py

from dg_maxwell import params
from dg_maxwell import msh_parser
from dg_maxwell import advection_2d
from dg_maxwell import global_variables as gvar
from dg_maxwell import utils
from dg_maxwell import advection_2d_arbit_mesh as a2d_arbit_mesh
from dg_maxwell import wave_equation_2d
from dg_maxwell import lagrange

af.set_backend(params.backend)
af.set_device(params.device)

plt.rcParams['figure.figsize']     = 12, 7.5
plt.rcParams['lines.linewidth']    = 1.5
plt.rcParams['font.family']        = 'serif'
plt.rcParams['font.weight']        = 'bold'
plt.rcParams['font.size']          = 20  
plt.rcParams['font.sans-serif']    = 'serif'
plt.rcParams['text.usetex']        = True
plt.rcParams['axes.linewidth']     = 1.5
plt.rcParams['axes.titlesize']     = 'medium'
plt.rcParams['axes.labelsize']     = 'medium'

plt.rcParams['xtick.major.size']   = 8
plt.rcParams['xtick.minor.size']   = 4
plt.rcParams['xtick.major.pad']    = 8
plt.rcParams['xtick.minor.pad']    = 8
plt.rcParams['xtick.color']        = 'k'
plt.rcParams['xtick.labelsize']    = 'medium'
plt.rcParams['xtick.direction']    = 'in'    

plt.rcParams['ytick.major.size']   = 8
plt.rcParams['ytick.minor.size']   = 4
plt.rcParams['ytick.major.pad']    = 8
plt.rcParams['ytick.minor.pad']    = 8
plt.rcParams['ytick.color']        = 'k'
plt.rcParams['ytick.labelsize']    = 'medium'
plt.rcParams['ytick.direction']    = 'in'
plt.rcParams['text.usetex']        = True
plt.rcParams['text.latex.unicode'] = True

gmshtranslator friendly reminder: rules and actions prototypes


def node_condition(tag,x,y,z,physgroups): 
def node_action(tag,x,y,z):
def element_condition(eletag,eletype,physgrp,nodes):
def element_action(eletag,eletype,physgrp,nodes):


In [2]:
params.mesh_file = '../read_and_plot_mesh/mesh/square_1_3.msh'
advec_var = gvar.advection_variables(params.N_LGL, params.N_quad,
                                     params.x_nodes, params.N_Elements,
                                     params.c, params.total_time, params.wave,
                                     params.c_x, params.c_y, params.courant,
                                     params.mesh_file, params.total_time_2d)
new_elements = msh_parser.rearrange_element_edges(advec_var.elements, advec_var)
advec_var.reassign_2d_elements(new_elements)

gmshTranslator: Initializing...
gmshTranslator: Mesh has 21 nodes.
gmshTranslator: Mesh has 15 elements.
gmshTranslator: Processed 40 lines.
gmshTranslator: There are 1 physical groups available: 
gmshTranslator:      > 0
gmshTranslator: Parsing nodes
gmshTranslator: Parsing elements
gmshTranslator: No rules for elements... skipping elements.
gmshTranslator: Parsing nodes
gmshTranslator: No rules for nodes... skipping nodes.
gmshTranslator: Parsing elements
advection_variables __init__ completed


/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1193: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)
/home/ubermensch/.local/anaconda3/lib/python3.6/site-packages/numpy/lib/polynomial.py:1220: FutureWarning: In the future extra properties will not be copied across when constructing one poly1d from another
  other = poly1d(other)


In [12]:
def contour_2d(x_mesh, y_mesh, u_mesh, index, advec_var):
    '''
    '''
    color_levels = np.linspace(0, 1.1, 100)

    x_mesh = np.array(x_mesh)
    y_mesh = np.array(y_mesh)
    u_mesh = np.array(u_mesh)

    fig = plt.figure()
    plt.contourf(x_mesh, y_mesh, u_mesh, 200,
                 levels = color_levels, cmap = 'jet')
    plt.gca().set_aspect('equal')
    plt.colorbar()
    plt.title('Time = %.2f' %(index * 2 * advec_var.delta_t_2d))
    fig.savefig('../../results/2D_Wave_images/%04d' %(index) + '.png', dpi = 300)
    plt.close('all')

    return

In [13]:
results_directory = '../../results/2D_Wave_images'

if not os.path.exists(results_directory):
    os.makedirs(results_directory)

# The directory where h5py files are stored.
h5py_directory = '../../results/2d_hdf5_%02d' %int(params.N_LGL)
path, dirs, files = os.walk(h5py_directory).__next__()
file_count = len(files)
print(h5py_directory)
print(file_count)

shape_u = utils.shape(advec_var.u_e_ij)

# Create a x-mesh
x_mesh = af.reorder(advec_var.x_e_ij, d0 = 0, d1 = 2, d2 = 1)
x_mesh = af.moddims(x_mesh, d0 = params.N_LGL,
                    d1 = params.N_LGL, d2 = shape_u[1])
x_mesh = af.moddims(x_mesh, d0 = params.N_LGL,
                    d1 = params.N_LGL * shape_u[1], d2 = 1)

# Create a y-mesh
y_mesh = af.reorder(advec_var.y_e_ij, d0 = 0, d1 = 2, d2 = 1)
y_mesh = af.moddims(y_mesh, d0 = params.N_LGL,
                    d1 = params.N_LGL, d2 = shape_u[1])
y_mesh = af.moddims(y_mesh, d0 = params.N_LGL,
                    d1 = params.N_LGL * shape_u[1], d2 = 1)

for i in np.arange(file_count):
    print(i)
    h5py_data = h5py.File('../../results/2d_hdf5_%02d/dump_timestep_%06d' % (int(params.N_LGL), int(i * 2)) + '.hdf5', 'r')
    u_LGL     = af.np_to_af_array(h5py_data['u_i'][:])
    u_LGL     = u_LGL[:, :, 0]
    u_mesh = af.reorder(u_LGL, d0 = 0, d1 = 2, d2 = 1)
    u_mesh = af.moddims(u_mesh, d0 = params.N_LGL,
                        d1 = params.N_LGL, d2 = shape_u[1])
    u_mesh = af.moddims(u_mesh, d0 = params.N_LGL,
                        d1 = params.N_LGL * shape_u[1], d2 = 1)

    contour_2d(x_mesh, y_mesh, u_mesh, i, advec_var)

../../results/2d_hdf5_08
285
0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
2